# Set up

In [1122]:
from worde4mde import load_embeddings
sgram_mde = load_embeddings('sgram-mde')

In [1123]:
word = 'ecore'
sgram_mde[word]

array([ 0.14674647,  0.42704162,  0.17717203,  0.05179158,  0.38020504,
       -0.00091264,  0.02094162,  0.00924711, -0.13806985,  0.1403339 ,
       -0.0953936 , -0.03681302,  0.38122246, -0.03746711, -0.17595871,
       -0.02031669,  0.23675247,  0.27676958, -0.07487358, -0.1987225 ,
        0.02015282,  0.05147164, -0.223504  ,  0.00699752, -0.06145973,
        0.00920826,  0.01364714,  0.10218145, -0.25330105, -0.14080754,
        0.2541444 ,  0.4577827 , -0.16825216, -0.08050938,  0.11404785,
       -0.38822263, -0.06395938, -0.4724689 , -0.00111705,  0.27869806,
        0.28032073,  0.02976846, -0.01675524,  0.09774698,  0.18298927,
       -0.24398401, -0.3238438 ,  0.01912955,  0.11861664,  0.08640955,
        0.18799016, -0.06891091, -0.03893414, -0.04862301,  0.02151174,
       -0.39205688, -0.23195332, -0.09532862, -0.01324714, -0.24555108,
        0.09372689, -0.21971853, -0.26171875, -0.3605202 , -0.07306654,
       -0.16288763,  0.12160088,  0.11416717,  0.17378834, -0.04

In [1124]:
# import pickle

import ast
import pandas as pd
import numpy as np

from worde4mde import load_embeddings
import re

import os
import openai
from openai import OpenAI
import json

import networkx as nx
import sys


In [1125]:
from dotenv import load_dotenv

# %%

load_dotenv()

openai_key = os.getenv('openai_key')
os.environ['OPENAI_API_KEY'] = openai_key

## Input

In [1126]:
##### update here to change input files

instructor_dir = "results/"

# 01_LabTracker
# 02_CelO
# 03_TSS
# 04_SHAS
# 05_OTS
# 06_Block
# 07_Tile
# 08_HBMS
problem_id = "08_HBMS"
instructor_input = instructor_dir + problem_id + ".txt"

llm_id = "gpt-4o-mini"
method_id = "sherpa"

student_dir = "results/" + llm_id + '/' + method_id + '/' + problem_id + '/'
if not os.path.exists(student_dir):
    # Create the directory
    os.makedirs(student_dir)
    
student_input = student_dir + problem_id + ".txt"
student_input
student_out_dir = student_dir + "result/"
student_out_dir

'results/gpt-4o-mini/sherpa/08_HBMS/result/'

In [1127]:
# First of all, you need to load the embeddings (currently supported: 'sgram-mde' and 'glove-mde')
sgram_mde = load_embeddings('sgram-mde')
# glove = load_embeddings('glove-mde')
# sgram_mde["id"]

In [1128]:
# word = 'id'
# sgram_mde.most_similar(positive=[word])

In [1129]:
def inEmbedding(word, embedding):
  try:
    return True, embedding[word]
  except:
    return False, None

In [1130]:

def splitCamelCase(word):
  splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', word)).split()
  return splitted

splitCamelCase("device ID")

['device', 'ID']

In [1131]:
splitCamelCase("CommandStatus")

['Command', 'Status']

In [1132]:
# client = OpenAI()

# def run_llm(prompt, model ="gpt-3.5-turbo"):
#   response = client.chat.completions.create(
#     model=model,
#     messages=[
#       {"role": "user", "content": prompt},

#     ]
#   )
#   return response.choices[0].message.content

In [1133]:
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [1134]:
def get_all_info(class_index, class_nodes, list_of_classes, list_edges, need_edge = True):
  result = ''
  result +=  list_of_classes[class_index] + "\n"
  node = class_nodes[class_index]
  if "abstract" in node:
      node = node.replace("abstract", "").strip()
  if "Abstract" in node:
      node = node.replace("Abstract", "").strip()
  if need_edge:
    for edge in list_edges:
      element = [i.strip() for i in edge.split()]
      if node in element:
        result += edge + "\n"

  return  result

In [1135]:
def match_classes(raw_1, raw_2, dict_attr, thresh = 0.5, verbose = False):
  # raw_1: nodes from reference solution
  # raw_2: nodes from student solution
  # dict_attr: dictionary for embeddings
  # thresh for subst cost
  # map class lists raw_1 and raw_2
  print(dict_attr)
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < thresh:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]
    


  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

# Group

In [1136]:
class Grader():
  def __init__(self):
    self.ref = None
    self.stu = None

In [1137]:
grader = Grader()

## Class

### Define metadata

In [1138]:
class ModelInfo():

  def __init(self):
    self.cls_atr = {}
    self.cls_name = []
    self.raw_dsl = []
    self.rel = None

In [1139]:
ref_cls = ModelInfo()
stu_cls = ModelInfo()

In [1140]:
grader.ref = ref_cls
grader.stu = stu_cls

In [1141]:
with open(instructor_input, 'r') as file:
    # Read the entire content of the file
    instructor_all = file.read()

with open(student_input, 'r') as file:
    # Read the entire content of the file
    student_all = file.read()

In [1142]:
print(instructor_all)

Enumerations:
RoomType(SINGLE, DOUBLE, TWIN)
BookingStatus(FINALIZED, CONFIRMED, CANCELLED_BY_HBMS, PRELIMINARY, CANCELLED_BY_TRAVELLER, CANCELLED_BY_HOTEL)
PaymentType (PREPAID, PAID_AT_HOTEL)

Classes:
HotelBookingManager()
HotelChain()

Traveller (string name, int reliabilityRating)

BillingInformation(string companyName, string address)
TravelPreference(string preference, string value)
Hotel (string city, string country, string area)
abstract Offer()
SpecialOffer()
RegularOffer()
TravelSearch()
SearchInfo()
RoomAvailability()
abstract TripInfo(int numberOfRooms, RoomType roomType, date arrivalDate, date departureDate, float budgetPerNight)
BookingInfo()
Booking (int bookingId, date cancellationDeadline, string creditCardNumber, BookingStatus bookingStatus, PaymentType paymentType, date confirmationDate, float paidAmount)


Relationships:



In [1143]:
print(student_all)

Enumerations:
BookingType(PrePaid, PayAtHotel)
RoomType(Single, Double, Twin)
HotelRating(OneStar, TwoStars, ThreeStars, FourStars, FiveStars)
TravelPreferenceType(BreakfastIncluded, FreeWifi, FrontDesk247)

Classes:

HotelChain(string name)  

Traveller(string name, BillingInformation billingInformation, ReliabilityRating reliabilityRating)  

BillingInformation(string billingAddress, string companyName)  

Hotel(string name, string address, string city, HotelChain hotelChain, int rating)  

abstract Booking(string bookingIdentifier, date dateArrival, date dateDeparture, int numberOfRooms, RoomType roomType, boolean isPrePaid, date cancellationDeadline)  
FinalizedBooking(boolean isConfirmed, date confirmationDeadline) inherit Booking  
AccommodationDeal(string price, string area, string keyPreferences)  
Address(string street, string city, string state, string zipCode)  
ReliabilityRating(int score)  
Competitor(string name, Offer offer)  
FinancialCompensation(string compensationDet

In [1144]:
shas_sol = instructor_all.split("Relationships:")[0]
stu_sol_g =  student_all.split("Relationships:")[0] # student solution

In [1145]:
ref_classes_raw = []
tmp = shas_sol.strip().splitlines()
ref_classes_raw = [i.strip() for i in tmp if len(i) > 0]
ref_classes_raw

reference_class = []
tmp = shas_sol.strip().splitlines()
reference_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]

In [1146]:
stu_classes_raw = []
tmp = stu_sol_g.strip().splitlines()
stu_classes_raw = [i.strip() for i in tmp if len(i) > 0]
stu_classes_raw

tmp = stu_sol_g.strip().splitlines()
stu_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]
stu_class

['Enumerations:',
 'BookingType',
 'RoomType',
 'HotelRating',
 'TravelPreferenceType',
 'Classes:',
 'HotelChain',
 'Traveller',
 'BillingInformation',
 'Hotel',
 'abstract Booking',
 'FinalizedBooking',
 'AccommodationDeal',
 'Address',
 'ReliabilityRating',
 'Competitor',
 'FinancialCompensation',
 'CreditCard']

In [1147]:
ref_attributes = {}
enum_index = ref_classes_raw.index("Enumerations:")
regular_index = ref_classes_raw.index("Classes:")
for cla, dsl in zip(reference_class, ref_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = ref_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"
    ref_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        ref_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

In [1148]:
reference_class.remove("Enumerations:")
reference_class.remove("Classes:")

ref_classes_raw.remove("Enumerations:")
ref_classes_raw.remove("Classes:")

In [1149]:
ref_cls.cls_name = reference_class
ref_cls.raw_dsl = ref_classes_raw
ref_cls.cls_atr = ref_attributes

In [1150]:
stu_attributes = {}
enum_index = stu_classes_raw.index("Enumerations:")
regular_index = stu_classes_raw.index("Classes:")

for cla, dsl in zip(stu_class, stu_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = stu_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"

    stu_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    print(dsl)
    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        stu_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

BookingType(PrePaid, PayAtHotel)
RoomType(Single, Double, Twin)
HotelRating(OneStar, TwoStars, ThreeStars, FourStars, FiveStars)
TravelPreferenceType(BreakfastIncluded, FreeWifi, FrontDesk247)
HotelChain(string name)
Traveller(string name, BillingInformation billingInformation, ReliabilityRating reliabilityRating)
BillingInformation(string billingAddress, string companyName)
Hotel(string name, string address, string city, HotelChain hotelChain, int rating)
abstract Booking(string bookingIdentifier, date dateArrival, date dateDeparture, int numberOfRooms, RoomType roomType, boolean isPrePaid, date cancellationDeadline)
FinalizedBooking(boolean isConfirmed, date confirmationDeadline) inherit Booking
AccommodationDeal(string price, string area, string keyPreferences)
Address(string street, string city, string state, string zipCode)
ReliabilityRating(int score)
Competitor(string name, Offer offer)
FinancialCompensation(string compensationDetails)
CreditCard(string cardNumber, string cardHo

In [1151]:
stu_class.remove("Enumerations:")
stu_class.remove("Classes:")

stu_classes_raw.remove("Enumerations:")
stu_classes_raw.remove("Classes:")

In [1152]:
stu_cls.cls_name = stu_class
stu_cls.raw_dsl = stu_classes_raw
stu_cls.cls_atr = stu_attributes

## Edges

### reference and student solutions

In [1153]:
shas_edges = instructor_all.split("Relationships:")[1]
shas_G2_edges =  student_all.split("Relationships:")[1] # student solution

In [1154]:
shas_edges

'\n'

In [1155]:
shas_edges

'\n'

In [1156]:
shas_edges = shas_edges.replace("0..*","*")

In [1157]:
shas_G2_edges = shas_G2_edges.replace("0..*","*")

In [1158]:
tmp = shas_edges.strip().splitlines()
ref_edges = [i.strip() for i in tmp if len(i) > 0]

for i in ref_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Reference Length error:", i)

tmp = shas_G2_edges.strip().splitlines()
stu_edges = [i.strip() for i in tmp if len(i) > 0]

for i in stu_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Student Length error:", i)



In [1159]:
class Relationship:
  def __init__(self):
    self.rels = [] # list of dict{}
    self.raw_dsl = []
  # find first non-matched or matched reltionshipi given dsl
  def find_relation(self, dsl, matched = False):
    for index, i in enumerate(self.rels):
      if i["dsl"] == dsl and i["counterpart"] is None:
        return index, i

    return None

In [1160]:
ref_edges_obj = Relationship()
stu_edges_obj = Relationship()


ref_edges_dict = []
for e in ref_edges:
  ref_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

ref_edges_obj.rels = ref_edges_dict
ref_edges_obj.raw_dsl = ref_edges

stu_edges_dict = []
for e in stu_edges:
  stu_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

stu_edges_obj.rels = stu_edges_dict
stu_edges_obj.raw_dsl = stu_edges

In [1161]:
# link to classInfo object
ref_cls.rel = ref_edges_obj
stu_cls.rel = stu_edges_obj

In [1162]:
edges = [ref_edges_obj, stu_edges_obj]

## Getting embedding

In [1163]:
def removeChars(text, chars):
    for c in chars:
        text = text.replace(c,"")
    return text


In [1164]:
def get_mde_embedding(text, embedding):
  # >>> get_mde_embedding("WhatDevice", sgram_mde)
  words = splitCamelCase(removeChars(text, ["(",")",","]))
  lowercase_list = [s.lower() for s in words]
  counter = 0
  emb = np.zeros(300)
  for w in lowercase_list:
    try:
      emb += embedding[w]
      counter += 1
    except:
      # if failed in emb the complete word, embed the token
      tmp = np.zeros(300)
      for char in list(w):

        tmp += embedding[char]
      emb += (tmp / len(w))
      counter += 1
  return emb / counter

In [1165]:
text = "AutomationStatus (Created, Edited, Activated, Deactivated)"
words = splitCamelCase(removeChars(text, ["(",")",","]))
words

['Automation', 'Status', 'Created', 'Edited', 'Activated', 'Deactivated']

In [1166]:
def cosine_distance(emb_i, emb_j):
  return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

## Stage 1.1 Class name mapping

In [1167]:
# get name embedding
# all info
similarity_mde = []
mde_embedding = sgram_mde
threshold = 0.7
similarity_mde_dsl = []
percentage = 0.8
for index, node in enumerate(ref_cls.cls_name):
  cls = node.split()[-1].strip() # get the class name, remove abstract key word
  
  mde_emb_i = get_mde_embedding(cls, mde_embedding)
  mde_emb_i_dsl = get_mde_embedding(ref_cls.cls_atr[node]["dsl"], mde_embedding)

  pair = []
  mde_pair = []
  for j, stu_node in enumerate(stu_cls.cls_name):
    cls = stu_node.split()[-1].strip() # get the class name, remove abstract key word
    print()
    mde_emb_j = get_mde_embedding(cls, mde_embedding)
    mde_emb_j_dsl = get_mde_embedding(stu_cls.cls_atr[stu_node]["dsl"], mde_embedding)

    mde_sim = cosine_distance(mde_emb_i, mde_emb_j)
    mde_sim_dsl = cosine_distance(mde_emb_i_dsl, mde_emb_j_dsl)

    mde_pair.append(mde_sim * percentage + (1-percentage) * mde_sim_dsl)

  # apply third quartile
  similarity_mde.append(mde_pair)

dict_sim_word = {}
for i in range(len(ref_cls.cls_name)):
  dict_sim_word[ref_cls.cls_name[i]] = {}
  for j in range(len(stu_cls.cls_name)):
    dict_sim_word[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_mde[i][j]

KeyError: "Key '2' not present"

In [1077]:
for key in dict_sim_word:
    exact_match = False
    exact_cls = ""
    for cls_2 in dict_sim_word[key]:
        if dict_sim_word[key][cls_2] > 0.99:
            exact_match = True
            exact_cls = cls_2
            print(cls_2)
            break
    
    if exact_match:    # keep the exact match, others to 0
        print("update")
        for cls_2 in dict_sim_word[key]:
            if cls_2 !=  exact_cls:
                dict_sim_word[key][cls_2] = 0

similarity_mde = []
for i in dict_sim_word:
    tmp = []
    for j in dict_sim_word[i]:
        tmp.append(dict_sim_word[i][j])

    similarity_mde.append(tmp)

In [ ]:
import pandas as pd

df = pd.DataFrame(index=reference_class, columns=stu_class, data=similarity_mde)
df

,ActionCardType,TileOApplication,abstract Game,Designer,Board,ConnectionPiece,Die,abstract Tile,ActionCard,ActionTile,Player,Turn
Mode,0.263548,0.204278,0.274989,0.140181,0.149749,0.234199,0.185278,0.148853,0.241467,0.197425,0.277882,0.269091
Color,0.198095,0.133178,0.186167,0.104607,0.146821,0.189357,0.092360,0.132587,0.096513,0.143394,0.237748,0.188706
TileO,0.303201,0.938481,0.236170,0.088312,0.249976,0.189412,0.181572,0.785995,0.248691,0.733542,0.233191,0.209227
Game,0.289456,0.280796,0.928925,0.252673,0.380028,0.266304,0.204554,0.305061,0.289216,0.375046,0.657997,0.246921
Connection,0.299397,0.231373,0.123969,0.143024,0.118273,0.703874,0.072788,0.148996,0.211142,0.214762,0.135394,0.149833
Die,0.102572,0.146695,0.176723,0.079455,0.115564,0.128325,0.835174,0.049701,0.100450,0.077862,0.160232,0.166951
Deck,0.192974,0.216692,0.167876,0.125694,0.147373,0.174352,0.145103,0.228037,0.163212,0.221843,0.180184,0.230796
abstract Tile,0.248523,0.705482,0.342603,0.177389,0.321670,0.196621,0.107394,0.959491,0.236785,0.859425,0.327007,0.246001
abstract ActionCard,0.885122,0.279548,0.313362,0.202667,0.271694,0.279779,0.130832,0.248392,0.925010,0.477821,0.347375,0.248928
RollDieActionCard,0.601770,0.281721,0.302956,0.197927,0.251918,0.249916,0.574259,0.192922,0.619365,0.356390,0.350590,0.295907


### Stage 1.1.1 Mapping (enum / regular + abstract)

In [1079]:
ref_enum = []
ref_reg_cls = []
stu_enum = []
stu_reg_cls = []

for enum in ref_cls.cls_name:
  if ref_cls.cls_atr[enum]["type"] == "enum":
    ref_enum.append(enum)
  else:
    ref_reg_cls.append(enum)

for enum in stu_cls.cls_name:
  if stu_cls.cls_atr[enum]["type"] == "enum":
    stu_enum.append(enum)
  else:
    stu_reg_cls.append(enum)

In [1080]:
enum_mapping = match_classes(ref_enum, stu_enum, dict_sim_word, thresh = 0.5)
enum_mapping[0]

{'Mode': {'ActionCardType': 0.2635481073141408, 'TileOApplication': 0.20427802430342534, 'abstract Game': 0.2749890665667466, 'Designer': 0.14018057615958093, 'Board': 0.14974862515593035, 'ConnectionPiece': 0.23419931425378707, 'Die': 0.18527809310768195, 'abstract Tile': 0.14885260212188722, 'ActionCard': 0.24146725079470344, 'ActionTile': 0.19742461852290527, 'Player': 0.27788228880700633, 'Turn': 0.2690913402865595}, 'Color': {'ActionCardType': 0.19809498250689417, 'TileOApplication': 0.13317766674284537, 'abstract Game': 0.18616688630105174, 'Designer': 0.10460733950580114, 'Board': 0.14682085478516266, 'ConnectionPiece': 0.18935660196302287, 'Die': 0.09236025602468209, 'abstract Tile': 0.13258747271542887, 'ActionCard': 0.09651307016285349, 'ActionTile': 0.14339428622113318, 'Player': 0.23774753813578048, 'Turn': 0.1887064406218897}, 'TileO': {'ActionCardType': 0.30320088987552485, 'TileOApplication': 0.9384811749161956, 'abstract Game': 0.2361699324443592, 'Designer': 0.08831193

[('Mode', None), ('Color', None), (None, 'ActionCardType')]

In [ ]:
cls_mapping = match_classes(ref_reg_cls, stu_reg_cls, dict_sim_word, thresh = 0.5)
cls_mapping[0]

{'Mode': {'ActionCardType': 0.2635481073141408, 'TileOApplication': 0.20427802430342534, 'abstract Game': 0.2749890665667466, 'Designer': 0.14018057615958093, 'Board': 0.14974862515593035, 'ConnectionPiece': 0.23419931425378707, 'Die': 0.18527809310768195, 'abstract Tile': 0.14885260212188722, 'ActionCard': 0.24146725079470344, 'ActionTile': 0.19742461852290527, 'Player': 0.27788228880700633, 'Turn': 0.2690913402865595}, 'Color': {'ActionCardType': 0.19809498250689417, 'TileOApplication': 0.13317766674284537, 'abstract Game': 0.18616688630105174, 'Designer': 0.10460733950580114, 'Board': 0.14682085478516266, 'ConnectionPiece': 0.18935660196302287, 'Die': 0.09236025602468209, 'abstract Tile': 0.13258747271542887, 'ActionCard': 0.09651307016285349, 'ActionTile': 0.14339428622113318, 'Player': 0.23774753813578048, 'Turn': 0.1887064406218897}, 'TileO': {'ActionCardType': 0.30320088987552485, 'TileOApplication': 0.9384811749161956, 'abstract Game': 0.2361699324443592, 'Designer': 0.08831193

[('TileO', 'TileOApplication'),
 ('Game', 'abstract Game'),
 ('Connection', 'ConnectionPiece'),
 ('Die', 'Die'),
 ('Deck', None),
 ('abstract Tile', 'abstract Tile'),
 ('abstract ActionCard', 'ActionCard'),
 ('RollDieActionCard', None),
 ('ConnectTilesActionCard', None),
 ('RemoveConnectionActionCard', None),
 ('TeleportActionCard', None),
 ('LoseTurnActionCard', 'Turn'),
 ('WinTile', None),
 ('ActionTile', 'ActionTile'),
 ('NormalTile', None),
 ('Player', 'Player'),
 (None, 'Designer'),
 (None, 'Board')]

In [1082]:
mapping = enum_mapping[0] + cls_mapping[0]

for map in mapping:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

In [1083]:
ref_cls.cls_atr

{'Mode': {'score': 0,
  'type': 'enum',
  'dsl': 'Mode(design, game)',
  'counterpart': None,
  'attributes': {'design': {'score': 0, 'counterpart': None},
   'game': {'score': 0, 'counterpart': None}}},
 'Color': {'score': 0,
  'type': 'enum',
  'dsl': 'Color(red, blue, green, yellow)',
  'counterpart': None,
  'attributes': {'red': {'score': 0, 'counterpart': None},
   'blue': {'score': 0, 'counterpart': None},
   'green': {'score': 0, 'counterpart': None},
   'yellow': {'score': 0, 'counterpart': None}}},
 'TileO': {'score': 1,
  'type': 'regular',
  'dsl': 'TileO()',
  'counterpart': 'TileOApplication',
  'attributes': {}},
 'Game': {'score': 0.5,
  'type': 'regular',
  'dsl': 'Game(int currentConnectionPieces, Mode mode, int spaceConnectionPieces, int numberOfActionCards)',
  'counterpart': 'abstract Game',
  'attributes': {'int currentConnectionPieces': {'score': 0,
    'counterpart': None},
   'Mode mode': {'score': 0, 'counterpart': None},
   'int spaceConnectionPieces': {'scor

## Stage 1.2 Class Mapping (with all info)

In [1084]:
ref_classes = []
stu_classes = []

for cls in ref_cls.cls_name:
  if ref_cls.cls_atr[cls]["counterpart"] == None:
    ref_classes.append(cls)

for cls in stu_cls.cls_name:
  if stu_cls.cls_atr[cls]["counterpart"] == None:
    stu_classes.append(cls)

In [1085]:
# get name embedding
# all info
similarity_all = []
mde_embedding = sgram_mde
threshold = 0.7



if os.path.exists(student_dir + "ada_embedding.txt"):
  with open(student_dir + "ada_embedding.txt", 'r') as json_file:
    dict_sim_all = json.load(json_file)

else:
  for index, node in enumerate(ref_cls.cls_name):
    text_1 = get_all_info(index, ref_cls.cls_name, ref_cls.raw_dsl, ref_cls.rel.raw_dsl, True)
    emb_i = get_embedding(text_1)
    mde_pair = []
    for j, stu_node in enumerate(stu_cls.cls_name):
      text_2 = get_all_info(j, stu_cls.cls_name, stu_cls.raw_dsl, stu_cls.rel.raw_dsl, True)
      emb_j = get_embedding(text_2)
      mde_sim = cosine_distance(emb_i, emb_j)
      mde_pair.append(mde_sim)

    # apply third quartile
    similarity_all.append(mde_pair)

  dict_sim_all = {}
  for i in range(len(ref_cls.cls_name)):
    dict_sim_all[ref_cls.cls_name[i]] = {}
    for j in range(len(stu_cls.cls_name)):
      dict_sim_all[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_all[i][j]





In [1086]:
with open(student_dir + "ada_embedding.txt", 'w') as json_file:
    json.dump(dict_sim_all, json_file)

In [1087]:
student_dir

'results/gpt-4o-mini/sherpa/07_Tile/'

In [1088]:
ref_classes

['Mode',
 'Color',
 'Deck',
 'RollDieActionCard',
 'ConnectTilesActionCard',
 'RemoveConnectionActionCard',
 'TeleportActionCard',
 'WinTile',
 'NormalTile']

In [1089]:
stu_classes

['ActionCardType', 'Designer', 'Board']

In [ ]:
dict_sim_all

{'Mode': {'ActionCardType': 0.7803789530484666,
  'TileOApplication': 0.7849335825989973,
  'abstract Game': 0.8203053596695445,
  'Designer': 0.8384511177977061,
  'Board': 0.8003124058499295,
  'ConnectionPiece': 0.7801746016658562,
  'Die': 0.7945335803096161,
  'abstract Tile': 0.7651446894110163,
  'ActionCard': 0.7659720074553982,
  'ActionTile': 0.7645772452203214,
  'Player': 0.7708272551342213,
  'Turn': 0.7752655725704595},
 'Color': {'ActionCardType': 0.752892455912234,
  'TileOApplication': 0.7617506888455718,
  'abstract Game': 0.7396951641901045,
  'Designer': 0.7864113579352578,
  'Board': 0.7777599471988865,
  'ConnectionPiece': 0.7691348153787241,
  'Die': 0.7895114818360068,
  'abstract Tile': 0.7434349500346253,
  'ActionCard': 0.741285088165323,
  'ActionTile': 0.7291483966884102,
  'Player': 0.7894573858569608,
  'Turn': 0.7504768283836784},
 'TileO': {'ActionCardType': 0.7832896941422128,
  'TileOApplication': 0.9242214417613405,
  'abstract Game': 0.7591889643075

In [ ]:
mapping = match_classes(ref_classes, stu_classes, dict_sim_all, 0.8)
mapping[0]

{'Mode': {'ActionCardType': 0.7803789530484666, 'TileOApplication': 0.7849335825989973, 'abstract Game': 0.8203053596695445, 'Designer': 0.8384511177977061, 'Board': 0.8003124058499295, 'ConnectionPiece': 0.7801746016658562, 'Die': 0.7945335803096161, 'abstract Tile': 0.7651446894110163, 'ActionCard': 0.7659720074553982, 'ActionTile': 0.7645772452203214, 'Player': 0.7708272551342213, 'Turn': 0.7752655725704595}, 'Color': {'ActionCardType': 0.752892455912234, 'TileOApplication': 0.7617506888455718, 'abstract Game': 0.7396951641901045, 'Designer': 0.7864113579352578, 'Board': 0.7777599471988865, 'ConnectionPiece': 0.7691348153787241, 'Die': 0.7895114818360068, 'abstract Tile': 0.7434349500346253, 'ActionCard': 0.741285088165323, 'ActionTile': 0.7291483966884102, 'Player': 0.7894573858569608, 'Turn': 0.7504768283836784}, 'TileO': {'ActionCardType': 0.7832896941422128, 'TileOApplication': 0.9242214417613405, 'abstract Game': 0.7591889643075148, 'Designer': 0.7693046200372154, 'Board': 0.80

[('Mode', 'Designer'),
 ('Color', None),
 ('Deck', 'Board'),
 ('RollDieActionCard', None),
 ('ConnectTilesActionCard', 'ActionCardType'),
 ('RemoveConnectionActionCard', None),
 ('TeleportActionCard', None),
 ('WinTile', None),
 ('NormalTile', None)]

In [1092]:
for map in mapping[0]:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

## Stage 2.1 Attribute mapping

In [1093]:
def get_cosine_disntance(text1, text2, embedding):
    avg = np.sum(embedding[text1], axis=0)
    emb_i = avg / len(text1)

    avg = np.sum(embedding[text2], axis=0)
    emb_j = avg / len(text2)

    return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

In [1094]:
def create_cosine_similarity_dict(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  dict_attr = {}
  for i in range(len(attr_1)):
    dict_attr[attr_1[i]] = {}
    for j in range(len(attr_2)):
      dict_attr[attr_1[i]][attr_2[j]] = similarities[i][j]

  return dict_attr

In [1095]:
def match_attributes(raw_1, raw_2, dict_attr, threshold = 0.5):
  # map attributes lists raw_1 and raw_2
  # >>> mapAttributes(['deviceStatus'], ['id'], dic)[0]
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < threshold:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]

  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

In [1096]:
def check_attributes_type(attr_1, attr_2, ref_dict, stu_dict):
  # given two attribute, compare whether their type can match
  attr_1 = attr_1.split() # from reference solution do not need to check it
  attr_2 = attr_2.split()

  # if enumerate literal and attributes
  # if len(attr_1) != len(attr_2):
  #   return 0.5, 0.5

  # if an attributes use a regular class as its type
  if len(attr_2) == 2:
    t = attr_2[0].strip()
    cls = stu_dict.get(t, None)
    if cls is not None and (stu_dict[t]['type'] == "regular" or stu_dict[t]['type'] == "abstract"):
      return 0.5, 0.5
    
  if len(attr_1) == 2 or len(attr_2) == 2:
    t = attr_1[0].strip()
    cls = ref_dict.get(t, None)
    # print(cls)
    t_2 = attr_2[0].strip()
    cls_2 = stu_dict.get(t_2, None)   
    # print(cls_2) 
    if cls is not None:
      if ref_dict[t]['counterpart'] != t_2:
        return 0.5, 0.5
    if cls_2 is not None:
      if stu_dict[t_2]['counterpart'] != t:
        return 0.5, 0.5
    
  return 1,1

In [1097]:
x =check_attributes_type(('CommandStatus commandStatus', 'ControllerStatus status')[0], ('CommandStatus commandStatus', 'ControllerStatus status')[1], ref_cls.cls_atr, stu_cls.cls_atr)
x

(1, 1)

### Stage 2.1.1 Attribute <-> Attribute (mapped classes)

In [ ]:
# map classes
pairs = []

# get matched pairs
for key in ref_cls.cls_atr:
  if ref_cls.cls_atr[key]["counterpart"] != None:
    pair = [key, ref_cls.cls_atr[key]["counterpart"]]
    pairs.append(pair)

for pair in pairs:
  if pair[0] and pair[1]:
    print(pair)

    # map attributes:
    # matched_ref = set()
    # matched_attr = set()

    raw_1 = []
    for attributes in ref_cls.cls_atr[pair[0]]["attributes"]:
      raw_1.append(attributes)

    raw_2 = []
    for attributes in stu_cls.cls_atr[pair[1]]["attributes"]:
      raw_2.append(attributes)

    cos_dict = create_cosine_similarity_dict(raw_1, raw_2, sgram_mde)
    mappings = match_attributes(raw_1, raw_2, cos_dict)[0]
    print("mapping", mappings)

    for mapping in mappings:
      if mapping[0] != None and mapping[1] != None:
        scores = check_attributes_type(mapping[0], mapping[1], ref_cls.cls_atr, stu_cls.cls_atr)
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]['score'] = scores[0]
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]["counterpart"] = (mapping[1], pair[1])
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]['score'] = scores[1]
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]["counterpart"] =  (mapping[0], pair[0])

    print("="*20)

['Mode', 'Designer']
mapping [('design', None), ('game', None), (None, 'string designerName')]
['TileO', 'TileOApplication']
mapping []
['Game', 'abstract Game']
mapping [('int currentConnectionPieces', None), ('Mode mode', None), ('int spaceConnectionPieces', None), ('int numberOfActionCards', 'boolean isActive'), (None, 'string gameName')]
['Connection', 'ConnectionPiece']
mapping [(None, 'string direction')]
['Die', 'Die']
mapping [(None, 'int sides')]
['Deck', 'Board']
mapping [(None, 'int width'), (None, 'int height')]
['abstract Tile', 'abstract Tile']
mapping [('int x', None), ('int y', None), ('boolean hasBeenVisited', 'boolean isVisited'), (None, 'string tileType')]
['abstract ActionCard', 'ActionCard']
mapping [('string instructions', 'string actionDescription')]
['ConnectTilesActionCard', 'ActionCardType']
mapping [(None, 'ExtraTurn'), (None, 'ConnectTiles'), (None, 'RemoveConnection'), (None, 'MovePiece'), (None, 'LoseTurn')]
['LoseTurnActionCard', 'Turn']
mapping [(None, '

### Stage 2.1.2 Attribute <-> Attribute (between any classes)

In [1099]:
def create_cosine_similarity_list(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  return similarities

In [1100]:
def combine_two_dict(list_1, list_2, atr_1, atr_2, cls_1, cls_2, weight_1 = 0.9, weight_2=0.1):
  # list_1: similarity between cls
  # list_2: cos similarity between attributes
  if weight_1 + weight_2 != 1:
    raise ValueError("weight_1 + weight_2 != 1")
  result = {}
  for i in range(len(atr_1)):
    atr_r = atr_1[i]
    cls_r = cls_1[i]
    tmp_ref = (atr_r, cls_r)
    result[tmp_ref] = {}
    for j in range(len(atr_2)):
      atr_s = atr_2[j]
      cls_s = cls_2[j]
      tmp_stu = (atr_s, cls_s)

      result[tmp_ref][tmp_stu] = weight_1 * list_1[i][j] + weight_2 * list_2[i][j]

  return result


In [ ]:
# attr between any classes

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get attributes on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)


    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 0.5)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 0.5)
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] =  mapping[0]

    if ref_cls.cls_atr[mapping[0][1]]['counterpart'] == mapping[1][1]:
      ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 1)

      stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 1)
print("="*20)

(('design', 'Mode'), None)
(('game', 'Mode'), ('string gameName', 'abstract Game'))
(('red', 'Color'), None)
(('blue', 'Color'), None)
(('green', 'Color'), None)
(('yellow', 'Color'), None)
(('int currentConnectionPieces', 'Game'), ('string currentPlayer', 'Turn'))
(('Mode mode', 'Game'), None)
(('int spaceConnectionPieces', 'Game'), ('int sides', 'Die'))
(('int x', 'abstract Tile'), ('string actionType', 'ActionTile'))
(('int y', 'abstract Tile'), ('int height', 'Board'))
(('int inactivityPeriod', 'ActionTile'), ('int width', 'Board'))
(('int turnsUntilActive', 'Player'), ('int turnNumber', 'Turn'))
(None, ('ExtraTurn', 'ActionCardType'))
(None, ('ConnectTiles', 'ActionCardType'))
(None, ('RemoveConnection', 'ActionCardType'))
(None, ('MovePiece', 'ActionCardType'))
(None, ('LoseTurn', 'ActionCardType'))
(None, ('string designerName', 'Designer'))
(None, ('string direction', 'ConnectionPiece'))
(None, ('string tileType', 'abstract Tile'))
(None, ('string playerName', 'Player'))


## Stage 2.2.1 Attribute mapping atr -> cls

In [1102]:
#  get non-mapped attributes on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
    if stu_cls.cls_atr[cls]["counterpart"] == None:
      raw_2.append(cls)
      stu_source.append(cls)
      tup_s.append((cls, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = (None, mapping[1][1])

    stu_cls.cls_atr[mapping[1][1]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["counterpart"] =  mapping[0]

print("="*20)


(('design', 'Mode'), None)
(('red', 'Color'), None)
(('blue', 'Color'), None)
(('green', 'Color'), None)
(('yellow', 'Color'), None)
(('Mode mode', 'Game'), None)


## Stage 2.2.2 Attribute mapping cls -> atr

In [1103]:
#  get non-mapped cls on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get class on instrucotr sides
for cls in ref_cls.cls_atr:
    if ref_cls.cls_atr[cls]["counterpart"] == None:
      raw_1.append(cls)
      ref_source.append(cls)
      tup_r.append((cls, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] = (None, mapping[0][1])

print("="*20)


(('Color', 'Color'), None)
(('RollDieActionCard', 'RollDieActionCard'), None)
(('RemoveConnectionActionCard', 'RemoveConnectionActionCard'), ('RemoveConnection', 'ActionCardType'))
(('TeleportActionCard', 'TeleportActionCard'), None)
(('WinTile', 'WinTile'), None)
(('NormalTile', 'NormalTile'), ('string tileType', 'abstract Tile'))
(None, ('ExtraTurn', 'ActionCardType'))
(None, ('ConnectTiles', 'ActionCardType'))
(None, ('MovePiece', 'ActionCardType'))
(None, ('LoseTurn', 'ActionCardType'))
(None, ('string designerName', 'Designer'))
(None, ('string direction', 'ConnectionPiece'))
(None, ('string playerName', 'Player'))


## Stage 3: Relationship mapping

In [1104]:
def check_relations_classes(ref_index, stu_index, ref_elements, stu_elements):
    # ref_index: index of the element in the dsl
    # stu_index: index of the element in the dsl
    # ref_elements: list[str] elements in the dsl
    # stu_elements: list[str] elements in the dsl

    ref_class_1 = ref_elements[ref_index]
    # check whether this is an abstract class
    dict_keys = list(ref_attributes.keys())
    if not ref_class_1 in dict_keys:
      abstract = "abstract " + ref_class_1
      ref_class_1 = abstract

    true_pair = ref_attributes[ref_class_1]["counterpart"]
    if isinstance(true_pair, tuple):
      boolean_1 = False
    elif true_pair == None:
      boolean_1 = False
    else:
      tmp = true_pair.replace("abstract ", "")
      boolean_1 = stu_elements[stu_index] == tmp
    return boolean_1

#### Compare rel

In [1105]:

def compare_edges(ref_e, stu_e):
  # compare if two relationships match
  # return bool, score, score

  ref_elements = ref_e.split()
  stu_elements = stu_e.split()
  # print(ref_elements, stu_elements)

  # exact match first
  # length must match
  n_ref = len(ref_elements)
  n_stu = len(stu_elements)
  if n_ref == n_stu and n_ref == 5:
    # all element must match
    boolean_1 = check_relations_classes(1,1,ref_elements,stu_elements )

    boolean_4 = check_relations_classes(4,4,ref_elements,stu_elements )

    # type
    boolean_2 = ref_elements[2] == stu_elements[2]

    # association
    boolean_0 = ref_elements[0] == stu_elements[0]

    # association
    boolean_3 = ref_elements[3] == stu_elements[3]

    if boolean_0 and boolean_1 and boolean_2 and boolean_3 and boolean_4:
      print("exact match success")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    boolean_1_4 = False
    boolean_4_1 = False
    boolean_1_4 = check_relations_classes(1,4,ref_elements,stu_elements )
    boolean_4_1 = check_relations_classes(4,1,ref_elements,stu_elements )

    # check if the relationship is fliped, this only works for associate, not containment
    if  "associate" in ref_elements[2] and "associate" in stu_elements[2]:
      # association
      boolean_0 = ref_elements[0] == stu_elements[3]

      # association
      boolean_3 = ref_elements[3] == stu_elements[0]

    if boolean_0 and boolean_1_4 and boolean_3 and boolean_4_1 and boolean_2:
      print("match success, flipped associate")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    # check if relationship is partially match
    if  (boolean_1 and boolean_4) or (boolean_1_4 and boolean_4_1):
      print("match partially success")
      print(ref_e, "|||", stu_e)
      return True, 0.5, 0.5

  if n_ref == n_stu and n_ref == 3:
    # all element must match
    boolean_0 = check_relations_classes(0,0,ref_elements,stu_elements )
    boolean_2 = check_relations_classes(2,2,ref_elements,stu_elements )

    # type
    boolean_1 = ref_elements[1] == stu_elements[1]

    if boolean_0 and boolean_1 and boolean_2:
      print("match success")
      print(ref_e, "|||", stu_e)
      # ref_dict[ref_e]["score"] = 1
      # ref_dict[ref_e]["counterpart"] = stu_e

      # stu_dict[stu_e]["score"] = 1
      # stu_dict[stu_e]["counterpart"] = ref_e
      return True, 1, 1


  return False,0, 0

In [1106]:

for i, ref_tmp in enumerate(edges[0].raw_dsl):
  matchings = []
  for j, stu_tmp in enumerate(edges[1].raw_dsl):
    if edges[0].rels[i]["counterpart"] is None and edges[1].rels[j]["counterpart"] is None:
      result = compare_edges(ref_tmp, stu_tmp)
      if result[0]:
        matching = (i, j, result[1], result[2])
        matchings.append(matching)
        # matched_stu_edges.add(j)
        # matched_ref_edges.add(i)
        # edges[0].rels[i]["score"] = result[1]
        # edges[0].rels[i]["counterpart"] = stu_tmp

        # edges[1].rels[j]["score"] = result[2]
        # edges[1].rels[j]["counterpart"] = ref_tmp
  print(matchings)
  if len(matchings) > 0:
    print()
    highest_score = 0
    matching = None

    for m in matchings:
      if m[2] > highest_score:
        matching = m
        highest_score = m[2]
    j_j = matching[1]
    i_i = matching[0]
    
    edges[0].rels[i_i]["score"] = m[2]
    edges[0].rels[i_i]["counterpart"] = edges[1].raw_dsl[j_j]

    edges[1].rels[j_j]["score"] = m[3]
    edges[1].rels[j_j]["counterpart"] = edges[0].raw_dsl[i_i]



## Result: Precision/Recall/F1

### save matching to disk

In [1107]:
if not os.path.exists(student_out_dir):
    os.makedirs(student_out_dir)

In [1108]:
# with open( student_out_dir + 'matching.pkl', 'wb') as outp:
#     pickle.dump(grader, outp, pickle.HIGHEST_PROTOCOL)

In [1109]:
student_out_dir

'results/gpt-4o-mini/sherpa/07_Tile/result/'

In [1110]:


# as requested in comment

with open(student_out_dir + 'solution_matching.txt', 'w') as file:
     file.write(json.dumps(grader.ref.cls_atr))


with open(student_out_dir + 'student_matching.txt', 'w') as file:
     file.write(json.dumps(grader.stu.cls_atr))

with open(student_out_dir + 'solution_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.ref.rel.rels))


with open(student_out_dir + 'student_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.stu.rel.rels))

In [1111]:
student_out_dir

'results/gpt-4o-mini/sherpa/07_Tile/result/'

###### to local

In [1112]:
# # # be careful...

# with open(student_dir + "human_eval/" + 'ref_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.ref.cls_atr))


# with open(student_dir + "human_eval/" + 'stu_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.stu.cls_atr))

# with open(student_dir + "human_eval/" + 'ref_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.ref.rel.rels))


# with open(student_dir + "human_eval/" + 'stu_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.stu.rel.rels))

In [1113]:
algo_result = {}
algo_result["class"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["attribute"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["relationship"] = {"precision": 0, "recall": 0, "f1":0}

### Class

In [1114]:
# recell
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  count += 1
  recall += ref_attributes[key]["score"]
print(count, recall)

algo_result["class"]['recall'] = recall / count

18 11.0


In [1115]:
# precision
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  count += 1
  precision += stu_attributes[key]["score"]

print(count, precision)

algo_result["class"]['precision'] = precision / count
algo_result["class"]['f1'] = 2* (algo_result["class"]['recall']   * algo_result["class"]['precision'] ) / ( algo_result["class"]['recall']   + algo_result["class"]['precision']  )


12 10.0


### Attribute

In [1116]:
# recall
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  attrs = ref_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    recall += ref_attributes[key]["attributes"][att]["score"]
print(count, recall)

algo_result["attribute"]['recall'] = recall / count

19 9.0


In [1117]:
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  attrs = stu_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    precision += stu_attributes[key]["attributes"][att]["score"]
print(count, precision)

algo_result["attribute"]['precision'] = precision / count

r = algo_result["attribute"]['recall']
p = algo_result["attribute"]['precision']

algo_result["attribute"]['f1'] = 2* (r * p) / (r + p)


22 10.0


### Relationship

In [1118]:
count = 0
recall = 0
for tmp in edges[0].rels:
  attrs = tmp["score"]

  count += 1
  recall += attrs
print(count, recall)

# algo_result["relationship"]['recall'] = recall / count

0 0


In [ ]:
count = 0
precision = 0
for tmp in edges[1].rels:
  attrs = tmp["score"]

  count += 1
  precision += attrs
print(count, precision)

# algo_result["relationship"]['precision'] = precision / count
r = algo_result["relationship"]['recall']
p = algo_result["relationship"]['precision']
# algo_result["relationship"]['f1'] = 2* (r * p ) / ( r + p )


0 0


In [1120]:

# as requested in comment

with open(student_out_dir + 'algo_result.txt', 'w') as file:
     file.write(json.dumps(algo_result))

In [1121]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

## Comparing with manual evaluation

In [92]:

class HumanEvaluation(object):
    def __init__(self, ref_cls=None, stu_cls=None, ref_rels=None, stu_rels=None):
        self.ref_cls = ref_cls
        self.stu_cls = stu_cls
        self.ref_rels = ref_rels
        self.stu_rels = stu_rels

group = HumanEvaluation()


In [ ]:


# as requested in comment

with open(student_dir + "human_eval/" + 'ref_meta_cls.py', 'r') as file:
    content = file.read()
    ref_cls_human = ast.literal_eval(content)
    group.ref_cls = ref_cls_human

with open(student_dir + "human_eval/" + 'stu_meta_cls.py', 'r') as file:
    content = file.read()
    stu_cls_human = ast.literal_eval(content)
    group.stu_cls = stu_cls_human

with open(student_dir + "human_eval/" + 'ref_meta_rels.py', 'r') as file:
    content = file.read()
    ref_rel_human = ast.literal_eval(content)
    group.ref_rels = ref_rel_human

with open(student_dir + "human_eval/" + 'stu_meta_rels.py', 'r') as file:
    content = file.read()
    stu_rel_human = ast.literal_eval(content)
    group.stu_rels = stu_rel_human

FileNotFoundError: [Errno 2] No such file or directory: 'results/gpt-4o-mini/sherpa/01_LabTracker/human_eval/ref_meta_cls.py'

### Save to disk

In [ ]:
# # # import pickle


# with open(student_dir + 'human_grade.pkl', 'wb') as outp:
#     g2 = HumanEvaluation(ref_cls_human, stu_cls_human, ref_rel_human, stu_rel_human)
#     pickle.dump(g2, outp, pickle.HIGHEST_PROTOCOL)




### Comparsion

In [ ]:
algo_human = {}
algo_human["class"] = {"precision": 0, "recall": 0, "f1":0}
algo_human["attribute"] = {"precision": 0, "recall": 0, "f1":0}
algo_human["relationship"] = {"precision": 0, "recall": 0, "f1":0}

#### Class

In [ ]:
def exist_mapping(mappings, one_mapping):
  for m in mappings:
    if m[0] == one_mapping[0] and m[1] == one_mapping[1]:
      return True # already exist
  return False

In [ ]:
# get human matching first:

tmp = group.ref_cls
human_mappings = []
for key in tmp:
  # print(tmp[key])
  human_mappings.append((key, tmp[key]["counterpart"], tmp[key]["score"]))

tmp = group.stu_cls
for key in tmp:
  # print(tmp[key])

  # reserve the order
  matching = (tmp[key]["counterpart"], key, tmp[key]["score"])

  if not exist_mapping(human_mappings, matching):
    human_mappings.append(matching)

In [ ]:
human_mappings

In [ ]:
# compare performance
# algo matching
grader.ref.cls_atr

tmp = grader.ref.cls_atr
EMB_mappings = []
for key in tmp:
  # print(tmp[key])
  EMB_mappings.append((key, tmp[key]["counterpart"], tmp[key]["score"]))

tmp = grader.stu.cls_atr

for key in tmp:
  # print(tmp[key])
  matching = (tmp[key]["counterpart"], key, tmp[key]["score"])

  if not exist_mapping(EMB_mappings, matching):
    EMB_mappings.append(matching)

In [ ]:
EMB_mappings

##### Start Evaluation

In [ ]:
# TP/FP/TN/FN
TP = []
FP = []
TN = []
FN = []

def check_TP_TN(map_human, map_algo):
  if map_human[0] == map_algo[0] and map_human[1] == map_algo[1] and not (None in map_human):
    return "TP"
  elif map_human[0] == map_algo[0] and map_human[1] == map_algo[1] and (None in map_human):
    return "TN"
  else:
    return "NA"

human_mappings_dict = {}
for i in human_mappings:
  human_mappings_dict[i] = False # not mapped
EMB_mappings_dict = {}
for i in EMB_mappings:
  EMB_mappings_dict[i] = False # not mapped

# filter out TP or TN first
for i in human_mappings:
  mappinged = False
  if not mappinged:
    for j in EMB_mappings:
      ans = check_TP_TN(i,j)
      if ans == "NA":
        continue
      elif ans == "TP":
        score = 1 if i[2] == j[2] else 0.5
        TP.append((i,j, score))
        human_mappings_dict[i] = True
        EMB_mappings_dict[j] = True
      elif ans == "TN":
        score = 1 if i[2] == j[2] else 0.5
        TN.append((i,j, score))
        human_mappings_dict[i] = True
        EMB_mappings_dict[j] = True


In [ ]:
def find_mapping_with_cls(mapping_dict, mapping, pos):
  # position = 0 or 1
  for i in mapping_dict:
    element = mapping[pos]

    if i[pos] == element:
      return i
  print("no mapping is found with", mapping, pos)
  return None


In [ ]:
human_mappings_dict

for i in human_mappings_dict:
  if human_mappings_dict[i]:
    continue # if mapped, ignore

    # i[0] is human pair
  mapping_0 = find_mapping_with_cls(EMB_mappings_dict, i, 0) # human result
  mapping_1 = find_mapping_with_cls(EMB_mappings_dict, i, 1) # human result
  if i[0] is not None and mapping_0:
    mapping = mapping_0
    # if mapping:
    if i[1] is not None and mapping[1] is not None: # a b	// a c	0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[1] is not None and mapping[1] is None: # a b	// a None	0	FN
      FN.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[1] is None and mapping[1] is not None: # a None // a b	0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

  elif i[1] is not None and mapping_1:
    mapping = mapping_1
    if i[0] is None and mapping[0] is not None: # none a //	b a 	0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[0] is not None and mapping[0] is None: # b a	// none a  	0	FN
      FN.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[0] is not None and mapping[0] is not None: # c a	// b a  0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True
  else:
    raise Exception(f"Sorry, no match found for {i}")



In [ ]:
EMB_mappings_dict

for i in EMB_mappings_dict:
  if EMB_mappings_dict[i]:
    continue # if mapped, ignore

    # i[0] is generaeted pair
  mapping_0 = find_mapping_with_cls(human_mappings_dict, i, 0) # human result
  mapping_1 = find_mapping_with_cls(human_mappings_dict, i, 1) # human result
  if i[0] is not None and mapping_0:
    mapping = mapping_0
    if i[1] is not None and mapping[1] is not None: # a b	// a c	0	FP
      FP.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[1] is not None and i[1] is None: # a b	// a None	0	FN
      FN.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[1] is None and i[1] is not None: # a None // a b	0	FP
      FP.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

  elif i[1] is not None and mapping_1:
    mapping = mapping_1
    if mapping[0] is not None and i[0] is None: # b a //	none a 	0	FN
      FN.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[0] is None and i[0] is not None: # none a //	b a  	0	FP
      FP.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[0] is not None and i[0] is not None: # c a	// b a  0	FP
      FP.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True
  else:
    raise Exception("matching failed")

In [ ]:
# precision = TP /(TP + FP)
# recall = TP /(TP + FN)

tp = 0

for i in TP:
  tp += i[2]

print("precision:", tp / (len(TP) + len(FP)))
print("recall:", tp / (len(TP) + len(FN)))

p = tp / (len(TP) + len(FP))
r = tp / (len(TP) + len(FN))

algo_human['class']["precision"] = p
algo_human['class']["recall"] = r

algo_human['class']["f1"] = 2 * p * r /  (p + r)


 a b => a c
 c

In [ ]:
for fp in FP:
    print(fp)

In [ ]:
for fn in FN:
    print(fn)

#### Attribute

In [ ]:
# algo matching

tmp = group.ref_cls
human_mappings = []
for cls  in tmp: # cls
  for attr in tmp[cls]["attributes"]: # attributes
    # print(cls, attr)

    human_mappings.append(((attr, cls), tmp[cls]["attributes"][attr]["counterpart"], tmp[cls]["attributes"][attr]["score"]))
    

tmp = group.stu_cls
for cls in tmp:
  # print(tmp[key])
  for attr in tmp[cls]["attributes"]: # attributes

    # order of the elements
    matching = (tmp[cls]["attributes"][attr]["counterpart"], (attr, cls), tmp[cls]["attributes"][attr]["score"])

    if not exist_mapping(human_mappings, matching):
      # print(matching)
      human_mappings.append(matching)

In [ ]:
human_mappings

In [ ]:
# compare performance
# algo matching
grader.ref.cls_atr

tmp = grader.ref.cls_atr
EMB_mappings = []
for cls in tmp:
  # print(tmp[key])
  for attr in tmp[cls]['attributes']:
    EMB_mappings.append(((attr, cls), tmp[cls]["attributes"][attr]["counterpart"], tmp[cls]["attributes"][attr]["score"]))

tmp = grader.stu.cls_atr

for cls in tmp:
  # print(tmp[key])
  for attr in tmp[cls]['attributes']:
    matching = (tmp[cls]["attributes"][attr]["counterpart"], (attr, cls),tmp[cls]["attributes"][attr]["score"])
    # if "LightController" in str(matching):
    #   print(matching)
    if not exist_mapping(EMB_mappings, matching):
      EMB_mappings.append(matching)

In [ ]:
# TP/FP/TN/FN
TP = []
FP = []
TN = []
FN = []

def check_TP_TN(map_human, map_algo):
  if map_human[0] == map_algo[0] and map_human[1] == map_algo[1] and not (None in map_human):
    return "TP"
  elif map_human[0] == map_algo[0] and map_human[1] == map_algo[1] and (None in map_human):
    return "TN"
  else:
    return "NA"

human_mappings_dict = {}
for i in human_mappings:
  human_mappings_dict[i] = False # not mapped
EMB_mappings_dict = {}
for i in EMB_mappings:
  EMB_mappings_dict[i] = False # not mapped

# filter out TP or TN first
for i in human_mappings:
  mappinged = False
  if not mappinged:
    for j in EMB_mappings:
      ans = check_TP_TN(i,j)
      if ans == "NA":
        continue
      elif ans == "TP":
        score = 1 if i[2] == j[2] else 0.5
        TP.append((i,j, score))
        human_mappings_dict[i] = True
        EMB_mappings_dict[j] = True
      elif ans == "TN":
        score = 1 if i[2] == j[2] else 0.5
        TN.append((i,j, score))
        human_mappings_dict[i] = True
        EMB_mappings_dict[j] = True


In [ ]:
human_mappings_dict

for i in human_mappings_dict:


  if human_mappings_dict[i]:
    continue # if mapped, ignore

  else:
    mapping_0 = find_mapping_with_cls(EMB_mappings_dict, i, 0) 
    mapping_1 = find_mapping_with_cls(EMB_mappings_dict, i, 1) 
    if i[0] is not None and mapping_0:
      mapping = mapping_0
      # print(i, mapping)
      if i[1] is not None and mapping[1] is not None: # a b	// a c	0	FP
        FP.append((i, mapping))
        human_mappings_dict[i] = True
        EMB_mappings_dict[mapping] = True

      elif i[1] is not None and mapping[1] is None: # a b	// a None	0	FN
        FN.append((i, mapping))
        human_mappings_dict[i] = True
        EMB_mappings_dict[mapping] = True

      elif i[1] is None and mapping[1] is not None: # a None // a b	0	FP
        FP.append((i, mapping))
        human_mappings_dict[i] = True
        EMB_mappings_dict[mapping] = True

    
    elif i[1] is not None and mapping_1:
      mapping = mapping_1 # generated pair
      # print(i)
      # print(mapping)
      if i[0] is None and mapping[0] is not None: # none a //	b a 	0	FP
        FP.append((i, mapping))
        human_mappings_dict[i] = True
        EMB_mappings_dict[mapping] = True

      elif i[0] is not None and mapping[0] is None: # b a	// none a  	0	FN
        FN.append((i, mapping))
        human_mappings_dict[i] = True
        EMB_mappings_dict[mapping] = True

      elif i[0] is not None and mapping[0] is not None: # c a	// b a  0	FP
        FP.append((i, mapping))
        human_mappings_dict[i] = True
        EMB_mappings_dict[mapping] = True



In [ ]:
EMB_mappings_dict

for i in EMB_mappings_dict:

  if EMB_mappings_dict[i]:
    continue # if mapped, ignore

    # i[0] is generaeted pair
  mapping_0 = find_mapping_with_cls(human_mappings_dict, i, 0) # human result
  mapping_1 = find_mapping_with_cls(human_mappings_dict, i, 1) # human result

  if i[0] is not None and mapping_0:
    # mapping = find_mapping_with_cls(human_mappings_dict, i, 0) # human result

    mapping = mapping_0
    if i[1] is not None and mapping_0[1] is not None: # a b	// a c	0	FP
      mapping = mapping_0
      FP.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[1] is not None and i[1] is None: # a b	// a None	0	FN
      FN.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[1] is None and i[1] is not None: # a None // a b	0	FP
      FP.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True


  elif i[1] is not None and mapping_1:
    mapping = mapping_1
    mapping = find_mapping_with_cls(human_mappings_dict, i, 1)
    if mapping[0] is not None and i[0] is None: # b a //	none a 	0	FN
      FN.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[0] is None and i[0] is not None: # none a //	b a  	0	FP
      FP.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[0] is not None and i[0] is not None: # c a	// b a  0	FP
      FP.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

In [ ]:
# precision = TP /(TP + FP)
# recall = TP /(TP + FN)

tp = 0

for i in TP:
  tp += i[2]

print("precision:", tp / (len(TP) + len(FP)))
print("recall:", tp / (len(TP) + len(FN)))

p = tp / (len(TP) + len(FP))
r = tp / (len(TP) + len(FN))

algo_human['attribute']["precision"] = p
algo_human['attribute']["recall"] = r

algo_human['attribute']["f1"] = 2 * p * r /  (p + r)

#### Relationship

In [ ]:
# algo matching

tmp = group.ref_rels

human_mappings = []
for dic  in tmp: # cls
  # cls is a dict
  
  matching = (dic['dsl'], dic['counterpart'], dic['score'] )

  human_mappings.append(matching)
    

tmp = group.stu_rels
for dic in tmp:

    # order of the elements
    matching = (dic['counterpart'], dic['dsl'], dic['score'])

    if not exist_mapping(human_mappings, matching):
      # print(matching)
      human_mappings.append(matching)

In [ ]:
# compare performance
# algo matching
grader.ref.rel.rels

tmp = grader.ref.rel.rels
EMB_mappings = []
for dic  in tmp: # cls
  # cls is a dict
  
  matching = (dic['dsl'], dic['counterpart'], dic['score'] )
  EMB_mappings.append(matching)

tmp = grader.stu.rel.rels

for dic in tmp:

    # order of the elements
    matching = (dic['counterpart'], dic['dsl'], dic['score'])

    if not exist_mapping(EMB_mappings, matching):
      # print(matching)
      EMB_mappings.append(matching)


##### Start evaluation

In [ ]:
# TP/FP/TN/FN
TP = []
FP = []
TN = []
FN = []

def check_TP_TN(map_human, map_algo):
  if map_human[0] == map_algo[0] and map_human[1] == map_algo[1] and not (None in map_human):
    return "TP"
  elif map_human[0] == map_algo[0] and map_human[1] == map_algo[1] and (None in map_human):
    return "TN"
  else:
    return "NA"

human_mappings_dict = {}
for i in human_mappings:
  human_mappings_dict[i] = False # not mapped
EMB_mappings_dict = {}
for i in EMB_mappings:
  EMB_mappings_dict[i] = False # not mapped

# filter out TP or TN first
for i in human_mappings:
  mappinged = False
  if not mappinged:
    for j in EMB_mappings:
      ans = check_TP_TN(i,j)
      if ans == "NA":
        continue
      elif ans == "TP":
        score = 1 if i[2] == j[2] else 0.5
        TP.append((i,j, score))
        human_mappings_dict[i] = True
        EMB_mappings_dict[j] = True
      elif ans == "TN":
        score = 1 if i[2] == j[2] else 0.5
        TN.append((i,j, score))
        human_mappings_dict[i] = True
        EMB_mappings_dict[j] = True


In [ ]:
human_mappings_dict

for i in human_mappings_dict:
  print(i)
  if human_mappings_dict[i]:
    continue # if mapped, ignore

    # i[0] is human pair
  elif i[0] is not None:
    mapping = find_mapping_with_cls(EMB_mappings_dict, i, 0) # generated pair
    if i[1] is not None and mapping[1] is not None: # a b	// a c	0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[1] is not None and mapping[1] is None: # a b	// a None	0	FN
      FN.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[1] is None and mapping[1] is not None: # a None // a b	0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

  elif i[1] is not None:
    mapping = find_mapping_with_cls(EMB_mappings_dict, i, 1) # generated pair
    if i[0] is None and mapping[0] is not None: # none a //	b a 	0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[0] is not None and mapping[0] is None: # b a	// none a  	0	FN
      FN.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True

    elif i[0] is not None and mapping[0] is not None: # c a	// b a  0	FP
      FP.append((i, mapping))
      human_mappings_dict[i] = True
      EMB_mappings_dict[mapping] = True



In [ ]:
EMB_mappings_dict

In [ ]:
EMB_mappings_dict

for i in EMB_mappings_dict:
  if EMB_mappings_dict[i]:
    continue # if mapped, ignore

    # i[0] is generaeted pair
  elif i[0] is not None:
    mapping = find_mapping_with_cls(human_mappings_dict, i, 0) # human result
    if i[1] is not None and mapping[1] is not None: # a b	// a c	0	FP
      FP.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[1] is not None and i[1] is None: # a b	// a None	0	FN
      FN.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[1] is None and i[1] is not None: # a None // a b	0	FP
      FP.append((mapping, i ))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

  elif i[1] is not None:
    mapping = find_mapping_with_cls(human_mappings_dict, i, 1)
    if mapping[0] is not None and i[0] is None: # b a //	none a 	0	FN
      FN.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[0] is None and i[0] is not None: # none a //	b a  	0	FP
      FP.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

    elif mapping[0] is not None and i[0] is not None: # c a	// b a  0	FP
      FP.append((i, mapping))
      human_mappings_dict[mapping] = True
      EMB_mappings_dict[i] = True

In [ ]:
# precision = TP /(TP + FP)
# recall = TP /(TP + FN)

tp = 0

for i in TP:
  tp += i[2]

print("precision:", tp / (len(TP) + len(FP)))
print("recall:", tp / (len(TP) + len(FN)))

p = tp / (len(TP) + len(FP))
r = tp / (len(TP) + len(FN))

algo_human['relationship']["precision"] = p
algo_human['relationship']["recall"] = r

algo_human['relationship']["f1"] = 2 * p * r /  (p + r)


In [ ]:
algo_human

In [ ]:
student_out_dir

In [ ]:

# as requested in comment

with open(student_out_dir + 'algo_evaluation.txt', 'w') as file:
     file.write(json.dumps(algo_human))